Universidade Federal de Alagoas

Instituto de Computação


Disciplina: Sistemas Distribuídos

Semestre letivo: 2018.2

Professor: André Lage Freitas


# Trabalho Prático AB2

**Importante**. Leia atentamente as especificações pois só serão aceitos trabalhos que as obedecerem.


## Parte 1: Implementação de um monitor de carga da trabalho descentralizado

**Implementar uma versão modificada do detector de falha não confiável** (_unreliable failure detector_) descrito na Seção _15.1.1 Failure assumptions and failure detectors_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

A **modificação** consiste em enviar informações sobre a carga do sistema (_system workload_) em vez de informações sobre falhas que indicariam os processos suspeitos e não-suspeitos. Para isso, em vez de cada processo enviar mensagem do tipo `I'm p and I'm alive`, os processos enviarão a medição interna da sua carga de trabalho. 

Para simular a medição da carga de trabalho dos processos, utilize números aleatórios entre `0` e `1` com precisão de uma casa decimal apenas, por exemplo, `0.4`.

Utilize como período de envio das mensagens `2` segundos.

### Detalhes de implementação

Cada processo do sistema deverá ser implementado como um Julia Worker. O Master será apenas o gestor do experimento, ou seja, será responsável para criar os processos (Julia Workers) e iniciar o sistema. Uma vez iniciado, o sistema deve funcionar por si sem depender do Master; uma vez que o algoritmo é descentralizado.

Para fins de visualização do monitoramento da carga de trabalho, os **Workers devem enviar ao Master** periodicamente (a cada `2` segundos) uma mensagem com o estado dos workers que eles monitoram (incluindo ele mesmo). Essa mensagem deve ser impresso na tela do master da seguinte forma.

```julia
[...]
Workload from Worker 4: [0.4,0.9,0.7]
Workload from Worker 2: [0.4,0.9,0.7]
Workload from Worker 3: [0.4,0.9,0.7]
[...]
```

A última linha acima significa que as cargas de trabalho dos Workers monitorados pelo Worker 3 são `0.4,0.9,0.7`, ou seja, os Workers têm as seguintes carga de trabalho:

* Worker 2 = 0.4
* Worker 3 = 0.9
* Worker 4 = 0.7

Lembre-se de que, em Julia, o ID do Master é 1 e os Workers têm ID maiores que 1.

## Parte 2: Implementação de um algoritmo de Eleição

**Implementar o algoritmo de Eleição** baseado em anel (_ring-based election algorithm_) descrito na Seção _15.3 Elections_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

Considere as seguintes condições.

1. Uma eleição nova é disparada **quando a carga de trabalho do líder for maior ou igual a `0.8`** e **o Worker que identificou essa alta carga de trabalho do líder tenha carga de trabalho menor que `0.8`**.
1. Em caso de empate, o Worker com ID mais alto ganha a eleição.
1. Caso não haja Worker com carga de trabalho menor que `0.8` para liderar, escolhe-se o Worker com menor carga de trabalho.

Quando um líder for eleito, ele deve enviar a a seguinte mensagem ao Master:

```julia
[...]
New leader elected! Worker 2
[...]
```


### Suporte de programação distribuída em Julia

Para utilizar o suporte de programação distribuída em Julia, primeiro devemos carergar a biblioteca `Distributed`:

```julia
using Pkg
Pkg.add("Distributed")
using Distributed
```

A seguir, segue uma lista (não extensiva) de funções que poderão ajudar na programação distribuída para a implementação do trabalho:

```julia
addprocs
remotecall
@fetchfrom
@everywhere
sleep
@spawn
```

Para buscar uma rápida ajuda sobre as funções, utilize `?` na frente da função (ver exemplo abaixo). 

```julia
?addprocs
```

Utilize o [material do mini-curso do Professor](https://github.com/proflage/2018-julia-hands-on) para estudo e a [documentação oficial da linguagem Julia (1.x)](https://docs.julialang.org/en/v1/) para demais dúvidas.

## Código-fonte

Documente seu código em comentários no próprio código-fonte.

**Todo o seu código tem que estar colado na próxima célula**, mesmo se ele tenha sido feito em IDEs, células separadas, etc. O Professor testará a apenas o que estiver colado na próxima célula.

In [1]:
# Espaço para escrever o código fonte.
using Pkg
Pkg.add("Distributed")
using Distributed

#TODO
addprocs(3)

# Valores iniciais
# Inicia a carga de trabalho com valor 0
@everywhere begin
    work_loads = fill(0.0, nprocs())
end

# O líder inicia com o numero de processos: 3
@everywhere begin
    leader = nprocs()
end

# candidato a líder
@everywhere begin
    leader_candidate = -1
end

# FUNÇÕES AUXILIARES
# Atualizando carga de trabalho do trabalhador de índice i
@everywhere function set_value_work_load(index, value_of_work_load)
    global work_loads[index] = value_of_work_load
end

# Atualizando processos
@everywhere function set_index_work_load(index)
    return (index == nprocs()) ? 2 : index + 1
end

# Alterando o líder
@everywhere function change_leader(leader_work_load)
    if(leader_candidate == 1)
        global leader_candidate = -1
        global leader = leader_work_load
    end
end

# Atualizando valores após a eleição
@everywhere function update_leader_value(leader_work_load)
    for i in workers()
        @spawnat i change_leader(leader_work_load)
    end
end

# Função para comparar valores
@everywhere function compare(a, b)
    if(a > b)
        return 1
    elseif(a < b)
        return 2
    else
        return 3
    end
end

# Função de eleição
@everywhere function function_of_election(index, current_work_load, current_leader)
    aux_param = set_index_work_load(index)

    global leader_candidate = 1
    # Confições de eleição
    if(index != current_leader)
        if(compare(work_loads[index], current_work_load) == 2)
            @spawnat aux_param function_of_election(aux_param, work_loads[index])
        elseif(work_loads[index] == current_work_load && inde == current_leader)
            @spawnat aux_param function_of_election(aux_param, current_work_load, index)
        else
            @spawnat aux_param function_of_election(aux_param, current_work_load, current_leader)
        end
    else
        @spawnat 1 println("New leader elected! Worker ", index)
        update_leader_value(index)
    end
end


# Checando a carga de trabalho para eleger o líder, conforme estabelecido
@everywhere function check_to_leader_election(index, current_work_load)
    for i in workers()
        if(@fetchfrom i leader_candidate == 1) 
            return 0
        end
    end
    # Condições de eleição
    if(work_loads[leader] >= 0.8 && work_loads[index] < 0.8 && leader_candidate == -1)
        global aux_param = set_index_work_load(index)
        global leader_candidate = 1
        @spawnat aux_param function_of_election(aux_param, current_work_load, index)
    end
end

# Dados de saída da carga de trabalho dos trabalhadores
@everywhere function print_data_work_load(index, aux_work_load)
    data = string("Workload from Worker ", index, ": [")
    for i in 2:nprocs()
        data = data * string(aux_work_load[i])# Concantena
        if( i <= nworkers())
            data = data * string(", ")# Concantena
        end
    end
    data = data * string("]")# Concatena
    return data
end

# Função principal
@everywhere function main_loop()
    index_i = Distributed.myid()
    while (true)
        values_of_work_load = round(rand(); digits = 1)# Carga de trabalho aleatória com valores entre [0;1]
        for i in workers()
            @spawnat i set_value_work_load(index_i, values_of_work_load)# Configurando carga de trabalho
        end
        sleep(2)# Sleep de 2 segundos
        remotecall(println, 1, print_data_work_load(index_i, work_loads))# Imprime os dados
        check_to_leader_election(index_i, work_loads[index_i])# Checa para eleger o líder
    end
end

# Printando o líder
println("LEADER ELECTION ALGORITHM!")
println("STARTING WORKERS!")
println("First leader: ", nprocs())

# Executando a main
for i in workers()
    @spawnat i main_loop()
end

 Resolving package versions...
  Updating `~/.julia/Project.toml`
 [no changes]
  Updating `~/.julia/Manifest.toml`
 [no changes]
LEADER ELECTION ALGORITHM!
STARTING WORKERS!
First leader: 4
Workload from Worker 4: [0.5, 0.8, 0.5]
Workload from Worker 2: [0.5, 0.8, 0.5]
Workload from Worker 3: [0.5, 0.8, 1.0]
New leader elected! Worker 2
Workload from Worker 4: [0.9, 0.4, 1.0]
Workload from Worker 2: [0.9, 0.4, 0.2]
Workload from Worker 3: [0.5, 0.4, 0.2]
Workload from Worker 4: [0.5, 0.1, 0.2]
Workload from Worker 2: [0.5, 0.1, 0.4]
Workload from Worker 3: [0.1, 0.1, 0.4]
Workload from Worker 4: [0.1, 0.6, 0.4]
Workload from Worker 2: [0.1, 0.6, 0.5]
Workload from Worker 3: [0.7, 0.6, 0.5]
Workload from Worker 4: [0.7, 0.1, 0.5]
Workload from Worker 2: [0.7, 0.1, 0.6]
Workload from Worker 3: [0.5, 0.1, 0.6]
Workload from Worker 4: [0.5, 0.1, 0.6]
Workload from Worker 2: [0.5, 0.1, 0.2]
Workload from Worker 3: [0.5, 0.1, 0.2]
Workload from Worker 4: [0.5, 0.6, 0.2]
Workload from Worker


## Entrega

As respostas deverão ser entregues na parte indicada desse arquivo reservada ao código fonte, no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl), que utiliza tecnologia [Jupyter](https://www.jupyter.org).

O programa deve ser implementado na linguagem de programação [**Julia** versão 1.x](https://julialang.org/downloads/). Seu trabalho será testado na [JuliaBox](https://juliabox.com). 

Baixe seu arquivo `.ipynb` e anexe-o ao Google Classroom. Não se esqueça de **testar seu arquivo `.ipynb` na JuliaBox** antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente através do Google Classroom**.



A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos pelo Professor.

### Prazos

O prazo de entrega está descrito no **Google Classroom**.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.